In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import os
import re
%matplotlib inline

# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
from rapidfuzz import fuzz, process, utils
import jellyfish
import snowflake.connector
# import torch
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('stsb-roberta-large')

# Imported the inaugural corpus from nltk.corpus
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import gensim
from gensim.utils import simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import num2words
from num2words import num2words
import time
# from time import time

pd.options.display.max_colwidth = 10000

In [12]:
cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_S', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')

In [13]:
query = '''SELECT DISTINCT DISTRIBUTOR_FTS_ID,DISTRIBUTOR_FdC_ID, 
DISTRIBUTOR_NAME,DISTRIBUTOR_ADDRESS, DISTRIBUTOR_STATE,DISTRIBUTOR_CITY,DISTRIBUTOR_ZIP FROM FDC_PROD_LIVE.FDC02.ENTITY_DISTRIBUTOR '''

In [14]:
distributor_list = pd.read_sql(query,cnx)

In [15]:
distributor_list = pd.read_sql(query,cnx).to_csv("~/Downloads/distributor_list.csv")

In [21]:
distributor_list = pd.read_csv("~/Downloads/distributor_list.csv", dtype='str')

In [22]:
len(distributor_list)

13005

In [23]:
distributor_list.isna().sum()

Unnamed: 0                0
DISTRIBUTOR_FTS_ID     5568
DISTRIBUTOR_FDC_ID        0
DISTRIBUTOR_NAME          0
DISTRIBUTOR_ADDRESS      98
DISTRIBUTOR_STATE         0
DISTRIBUTOR_CITY         76
DISTRIBUTOR_ZIP         114
dtype: int64

In [24]:
distributor_list[distributor_list["DISTRIBUTOR_FTS_ID"].isna()]

,Unnamed: 0,DISTRIBUTOR_FTS_ID,DISTRIBUTOR_FDC_ID,DISTRIBUTOR_NAME,DISTRIBUTOR_ADDRESS,DISTRIBUTOR_STATE,DISTRIBUTOR_CITY,DISTRIBUTOR_ZIP
749,749,NaN,11926,BEVERAGE CONTROL INC,4333 EDINGTON RD,TN,KNOXVILLE,37920
788,788,NaN,12288,SPECIALTY BEVERAGE - VA,1400 COMMERCE RD,VA,RICHMOND,23224
1130,1130,NaN,16557,I.H. CAFFEY DIST. - CONCORD,151 ODELL SCHOOL RD,NC,CONCORD,28027
1421,1421,NaN,15595,AMOSKEAG BEVERAGES INC.,510 HALL ST,NH,BOW,03304
2232,2232,NaN,29320,ROHLFING OF BRAINERD INC.,923 WRIGHT ST,MN,BRAINERD,56401
...,...,...,...,...,...,...,...,...
12359,12359,NaN,8407,VIRGINIA IMPORTS LTD.,7550 ACCOTINK PARK RD,VA,SPRINGFIELD,22150
12425,12425,NaN,86,TREU HOUSE OF MUNCH INC.,8000 ARBOR DR,OH,NORTHWOOD,43619
12712,12712,NaN,8973,BLUE RIBBON PRODUCTS CO.,2410 MCDONOUGH ST,IL,JOLIET,60436
12744,12744,NaN,9089,PEERLESS BEVERAGE,1000 FLORAL AVE,NJ,UNION,07083


In [25]:
distributor_list[distributor_list['DISTRIBUTOR_FTS_ID'] !=distributor_list['DISTRIBUTOR_FDC_ID'] ]

,Unnamed: 0,DISTRIBUTOR_FTS_ID,DISTRIBUTOR_FDC_ID,DISTRIBUTOR_NAME,DISTRIBUTOR_ADDRESS,DISTRIBUTOR_STATE,DISTRIBUTOR_CITY,DISTRIBUTOR_ZIP
43,43,16082,10310,"FRESH BEER, LLC",1520 SE 3RD AVE,FL,FORT LAUDERDALE,33316
81,81,32179,102471,PLANET WINE - DISABLED,44 GOUGH ST,CA,SAN FRANCISCO,94103
89,89,58962,103449,BUSHNELL CRAFT LLC - DISABLED,8461 164TH AVE NE,WA,REDMOND,98052
93,93,134284,104770,WORLD TRADE LIQUOR IMPORTS,13713 N UNITEC DR,TX,LAREDO,78045
116,116,81438,101683,MATCHLESS BREWING - INACTIVE,8036 RIVER DR SE,WA,TUMWATER,98501
...,...,...,...,...,...,...,...,...
12712,12712,NaN,8973,BLUE RIBBON PRODUCTS CO.,2410 MCDONOUGH ST,IL,JOLIET,60436
12744,12744,NaN,9089,PEERLESS BEVERAGE,1000 FLORAL AVE,NJ,UNION,07083
12804,12804,15158,93538,STELLAR WINES GA,338 3RD ST,GA,ROME,30161
12979,12979,122011,99824,YACHATS BREWING + FARMSTORE,348 US HWY 101,OR,YACHATS,97498


In [26]:
# distributor_list.dropna(inplace=True)

In [27]:
distributor_list["dist_name_address"] = distributor_list['DISTRIBUTOR_NAME'] + " " + distributor_list['DISTRIBUTOR_ADDRESS'] + " " + \
distributor_list['DISTRIBUTOR_STATE'] + " " +  distributor_list['DISTRIBUTOR_CITY'] +" " +  distributor_list['DISTRIBUTOR_ZIP'].map(str)

In [28]:
distributor_list["dist_address"] = distributor_list['DISTRIBUTOR_ADDRESS'] + " " + \
distributor_list['DISTRIBUTOR_STATE'] + " " +  distributor_list['DISTRIBUTOR_CITY'] +" " +  distributor_list['DISTRIBUTOR_ZIP'].map(str)

In [29]:
type(distributor_list['dist_name_address'])

pandas.core.series.Series

In [30]:
distributor_list.dist_name_address.fillna(distributor_list.dist_name_address, inplace = True)
distributor_list.dist_address.fillna(distributor_list.dist_address, inplace = True)


In [31]:
def fuzz_match(x,y):
    return fuzz.partial_ratio(x,y)

In [32]:
distributor_list.apply(fuzz_match(distributor_list.dist_name_address,distributor_list.dist_address))

TypeError: s1 must be a String

In [35]:
distributor_list["fuzz_partial_ratio"]  = distributor_list.apply(lambda x: fuzz.partial_ratio(x.dist_name_address, x.dist_address), axis=1)

TypeError: s1 must be a String

In [33]:
distributor_list["fuzz_partial_ratio"].value_counts()


KeyError: 'fuzz_partial_ratio'